In [1]:
import pandas as pd

df = pd.read_excel("data\kospidaq_embeddings.xlsx")
df

,date,ticker,broker,target_price,rating,content,embedding_1,embedding_2,embedding_3,embedding_4,...,embedding_1527,embedding_1528,embedding_1529,embedding_1530,embedding_1531,embedding_1532,embedding_1533,embedding_1534,embedding_1535,embedding_1536
0,2025-10-10,현대모비스,한화투자증권,"360,000",Buy,[3Q25 Preview] 관세 영향에도 안정적 3Q 매출은 모듈/핵심부품과 A/S...,-0.001488,-0.009933,0.009638,-0.035406,...,-0.015297,-0.024625,0.033820,0.019464,0.021950,-0.002102,-0.017972,0.015727,0.002831,-0.001401
1,2025-10-10,대웅제약,DS투자증권,"210,000",매수,"3Q25 Pre: 비만패치제, 본 궤도 탑승 별도기준 대웅제약의 3Q25 매출액 및...",-0.006253,-0.019637,0.020031,-0.025595,...,-0.012112,-0.007118,0.046766,0.008441,0.018212,0.015756,-0.015512,0.013673,0.000746,-0.019257


In [4]:
df.dtypes

date               object
ticker             object
broker             object
target_price       object
rating             object
                   ...   
embedding_1532    float64
embedding_1533    float64
embedding_1534    float64
embedding_1535    float64
embedding_1536    float64
Length: 1542, dtype: object

# lamda 확인

In [1]:
import numpy as np
import pandas as pd

# -----------------------------
# 설정
# -----------------------------
KOSPI_PATH = r"data\kospi_krx_20130102_20251230.xlsx"
RF_PATH    = r"data\treasury3m_proxy_krx_20130102_20251230.xlsx"

RISK_FREE_TRADING_DAYS = 252
MIN_OBS_Q = 1    # 분기 최소 관측일
MIN_OBS_Y = 1   # 연도 최소 관측일
LAMBDA_FLOOR = None
LAMBDA_CAP   = None

# -----------------------------
# 로드
# -----------------------------
df_kospi = pd.read_excel(KOSPI_PATH, index_col=0)
df_rf    = pd.read_excel(RF_PATH, index_col=0)

df_kospi.index = pd.to_datetime(df_kospi.index)
df_rf.index    = pd.to_datetime(df_rf.index)

df_kospi = df_kospi.sort_index()
df_rf    = df_rf.sort_index()

# KOSPI 종가 컬럼 자동 선택
mkt_col_candidates = ["close", "CLSPRC_IDX", "Close", "종가"]
mkt_col = next((c for c in mkt_col_candidates if c in df_kospi.columns), None)
if mkt_col is None:
    raise ValueError(f"KOSPI 종가 컬럼을 찾지 못했습니다. columns={list(df_kospi.columns)[:10]}...")

# RF 컬럼 자동 선택
rf_col = "yield_3m_proxy" if "yield_3m_proxy" in df_rf.columns else df_rf.columns[0]

# -----------------------------
# 일별 수익률/초과수익률
# -----------------------------
mkt_ret  = df_kospi[mkt_col].astype(float).pct_change().rename("mkt_ret")
rf_daily = (pd.to_numeric(df_rf[rf_col], errors="coerce") / 100.0 / RISK_FREE_TRADING_DAYS).rename("rf_daily")

risk_df = pd.concat([mkt_ret, rf_daily], axis=1, join="inner").dropna()
risk_df["excess_ret"] = risk_df["mkt_ret"] - risk_df["rf_daily"]

# 전체(글로벌) lambda: 결측 period fallback용
global_lambda = risk_df["excess_ret"].mean() / risk_df["mkt_ret"].var()
if not np.isfinite(global_lambda):
    global_lambda = 2.5

# -----------------------------
# period별 lambda 계산 함수
# lambda = mean(excess_ret) / var(mkt_ret)
# -----------------------------
def summarize_lambda_by_period(df, freq, min_obs, floor=0.1, cap=20.0):
    rows = []
    for period_end, g in df.groupby(pd.Grouper(freq=freq)):
        if g.empty:
            continue

        n_obs = len(g)
        mean_excess = g["excess_ret"].mean()
        var_mkt = g["mkt_ret"].var(ddof=1)

        if (n_obs < min_obs) or (not np.isfinite(var_mkt)) or (var_mkt <= 0):
            lam_raw = np.nan
        else:
            lam_raw = mean_excess / var_mkt

        rows.append({
            "period_end": period_end,
            "n_obs": n_obs,
            "mean_excess": mean_excess,
            "var_mkt": var_mkt,
            "lambda_raw": lam_raw
        })

    out = pd.DataFrame(rows).set_index("period_end").sort_index()

    # fallback + clipping
    out["lambda"] = out["lambda_raw"].fillna(global_lambda)
    out["lambda"] = out["lambda"].clip(lower=floor, upper=cap)

    return out

# 분기/연도별
lambda_quarterly = summarize_lambda_by_period(
    risk_df, freq="Q", min_obs=MIN_OBS_Q, floor=LAMBDA_FLOOR, cap=LAMBDA_CAP
)
lambda_yearly = summarize_lambda_by_period(
    risk_df, freq="Y", min_obs=MIN_OBS_Y, floor=LAMBDA_FLOOR, cap=LAMBDA_CAP
)

# 보기 좋게 인덱스 라벨
lambda_quarterly.index = lambda_quarterly.index.to_period("Q")
lambda_yearly.index    = lambda_yearly.index.to_period("Y")

print("=== Quarterly lambda ===")
print(lambda_quarterly[["n_obs", "lambda_raw", "lambda"]].tail(12))
print("\n=== Yearly lambda ===")
print(lambda_yearly[["n_obs", "lambda_raw", "lambda"]].tail(10))

# 필요 시 저장
lambda_quarterly.to_csv(r"backtest\lambda_quarterly.csv", encoding="utf-8-sig")
lambda_yearly.to_csv(r"backtest\lambda_yearly.csv", encoding="utf-8-sig")


C:\Users\82105\AppData\Local\Temp\ipykernel_8184\1166482878.py:57: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  for period_end, g in df.groupby(pd.Grouper(freq=freq)):
C:\Users\82105\AppData\Local\Temp\ipykernel_8184\1166482878.py:57: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  for period_end, g in df.groupby(pd.Grouper(freq=freq)):


=== Quarterly lambda ===
            n_obs  lambda_raw     lambda
period_end                              
2023Q1         62   15.961546  15.961546
2023Q2         61   10.416978  10.416978
2023Q3         62  -10.583562 -10.583562
2023Q4         60    6.961786   6.961786
2024Q1         61    4.453634   4.453634
2024Q2         60    2.064146   2.064146
2024Q3         62   -4.469360  -4.469360
2024Q4         61  -10.270901 -10.270901
2025Q1         58    4.205381   4.205381
2025Q2         60   16.146108  16.146108
2025Q3         65   15.180427  15.180427
2025Q4         59   12.161020  12.161020

=== Yearly lambda ===
            n_obs  lambda_raw     lambda
period_end                              
2016          246    1.744095   1.744095
2017          243   22.437021  22.437021
2018          244   -9.805712  -9.805712
2019          246    4.229167   4.229167
2020          248    3.796613   3.796613
2021          248    1.527682   1.527682
2022          246   -8.684011  -8.684011
2023     

In [3]:
import numpy as np
import pandas as pd

# =============================
# Expanding Lambda (No fallback / No clipping)
# =============================

# 1) 설정
KOSPI_PATH = r"data\kospi_krx_20130102_20251230.xlsx"
RF_PATH    = r"data\treasury3m_proxy_krx_20130102_20251230.xlsx"

RISK_FREE_TRADING_DAYS = 252
MIN_OBS = 60  # 표본 충분 조건

# 2) 데이터 로드
df_kospi = pd.read_excel(KOSPI_PATH, index_col=0)
df_rf    = pd.read_excel(RF_PATH, index_col=0)

df_kospi.index = pd.to_datetime(df_kospi.index)
df_rf.index    = pd.to_datetime(df_rf.index)

df_kospi = df_kospi.sort_index()
df_rf    = df_rf.sort_index()

# 3) 컬럼 자동 선택
mkt_col_candidates = ["close", "CLSPRC_IDX", "Close", "종가"]
mkt_col = next((c for c in mkt_col_candidates if c in df_kospi.columns), None)
if mkt_col is None:
    raise ValueError(f"KOSPI 종가 컬럼을 찾지 못했습니다. columns={list(df_kospi.columns)[:10]}...")

rf_col = "yield_3m_proxy" if "yield_3m_proxy" in df_rf.columns else df_rf.columns[0]

# 4) 일별 수익률/무위험수익률
mkt_ret = df_kospi[mkt_col].astype(float).pct_change().rename("mkt_ret")
rf_daily = (pd.to_numeric(df_rf[rf_col], errors="coerce") / 100.0 / RISK_FREE_TRADING_DAYS).rename("rf_daily")

# 5) 정렬 및 초과수익률
risk_df = pd.concat([mkt_ret, rf_daily], axis=1, join="inner").dropna()
risk_df["excess_ret"] = risk_df["mkt_ret"] - risk_df["rf_daily"]

# 6) Expanding lambda 계산
#    lambda_t = expanding_mean(excess_ret) / expanding_var(mkt_ret)
mean_excess_exp = risk_df["excess_ret"].expanding(min_periods=MIN_OBS).mean()
var_mkt_exp     = risk_df["mkt_ret"].expanding(min_periods=MIN_OBS).var(ddof=1)

lambda_exp = (mean_excess_exp / var_mkt_exp).replace([np.inf, -np.inf], np.nan)
lambda_exp.name = "lambda"

# 7) 유효 구간만 사용 (왜곡 방지: ffill/fallback/clip 없음)
lambda_exp_valid = lambda_exp.dropna()

# 8) 결과 테이블
lambda_exp_df = pd.concat(
    [
        risk_df[["mkt_ret", "rf_daily", "excess_ret"]],
        mean_excess_exp.rename("mean_excess_exp"),
        var_mkt_exp.rename("var_mkt_exp"),
        lambda_exp,
    ],
    axis=1
)

print("전체 일수:", len(lambda_exp))
print("유효 lambda 일수:", len(lambda_exp_valid))
print("\n초기 유효 구간:")
print(lambda_exp_valid.head())
print("\n최근 구간:")
print(lambda_exp_valid.tail())

# 9) 저장
lambda_exp_df.to_csv(r"backtest\lambda_expanding_daily_full.csv", encoding="utf-8-sig")
lambda_exp_valid.to_csv(r"backtest\lambda_expanding_daily_valid.csv", encoding="utf-8-sig")


전체 일수: 3110
유효 lambda 일수: 3051

초기 유효 구간:
date
2013-07-09   -12.780525
2013-07-10   -13.434466
2013-07-11    -6.704761
2013-07-12    -7.366833
2013-07-15    -6.925015
Name: lambda, dtype: float64

최근 구간:
date
2025-12-23    1.998801
2025-12-24    1.992211
2025-12-26    2.006831
2025-12-29    2.068416
2025-12-30    2.063610
Name: lambda, dtype: float64
